## Helper functions

In [33]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy import stats
from IPython.display import display, Markdown, Latex

In [ ]:
def print_stats(values):
    mean = values.mean()
    variance = values.var()

In [91]:
def display_results(hypothesis: str, test: str, critical_values: str, test_statistic: str, conclusion: str):
    display(Markdown(f"**Hypothesis:** {hypothesis}\n\n**Suitable Test:** {test}\n\n**Critical value(-s):** {critical_values}\n\n**Test statistic:** {test_statistic}\n\n**Conclusion:** {conclusion}"))

In [134]:
def critical_values_string(critical_values):
    assert 0 < len(critical_values) <= 2
    if len(critical_values) == 1:
        return f"$C = {critical_values[0]}$"
    else:
        return f"$C_1 = {critical_values[0]}, C_2 = {critical_values[1]}$"

In [133]:
def conclude(critical_values, test_statistic: float, two_tailed: bool = True, smaller_than: bool = None):
    assert not two_tailed or (smaller_than == None), "You cannot do a two tailed t-test and provide a value for the 'smaller_than' parameter"
    assert two_tailed or (smaller_than != None), "You have to either set the 'two_tailed' parameter to true or provide the 'smaller_than' parameter"
    if two_tailed:
        assert len(critical_values) == 2
        assert critical_values[0] < critical_values[1]
        if critical_values[0] <= test_statistic <= critical_values[1]:
            return f"Since $C_1 = {critical_values[0]} \\leq {test_statistic} \\leq {critical_values[1]} = C_2$ we cannot reject $H_0$"
        elif test_statistic < critical_values[0] or test_statistic > critical_values[1]:
            return f"Reject $H_0$ because ${test_statistic} (\\lt|\\gt) ({critical_values[0]}|{critical_values[1]}) = (C_1|C_2)$"
    else:
        assert len(critical_values) == 1
        c = critical_values[0]
        if smaller_than:
            if test_statistic >= c:
                return f"Since ${test_statistic} \\nless {c} = C$ we cannot reject $H_0$"
            else:
                return f"Reject $H_0$ because $t = {test_statistic} \\lt {c} = C$"
        else:
            if test_statistic <= c:
                return f"Since ${test_statistic} \\ngtr {c} = C$ we cannot reject $H_0$"
            else:
                return f"Reject $H_0$ because $t = {test_statistic} \\gt {c} = C$"

## One sample t-test
Objective: Test hypothesis about the population mean $\mu$ based on one sample

Key questions:
    
- Two-tailed: Does the population mean $\mu$ deviate from a hypothesized mean $\mu_0$?
    
- One-tailed: Is the population mean $\mu$ smaller or larger than a hypothesized mean $\mu_0$?
    
Hypotheses (two-tailed):

- $H_0: \mu = \mu_0$

- $H_1: \mu \neq \mu_0$

Test statistic: $t=\frac{\bar{x}-\mu_0}{s}*\sqrt{n}$ , with sample mean $\bar{x}$, hypothesized mean $\mu_0$ and sample standard deviation $s$

If $H_0$ is true, the test statistic follows a t distribution with $df = n - 1$

In [144]:
def one_sample_t_test(x: np.ndarray, hypothesized_mean: float, two_tailed: bool = True, smaller_than: bool = None, alpha: float = 0.05):
    assert 0 < alpha < 1, "Aplha has to be between 0 and 1"
    assert not two_tailed or (smaller_than == None), "You cannot do a two tailed t-test and provide a value for the 'smaller_than' parameter"
    assert two_tailed or (smaller_than != None), "You have to either set the 'two_tailed' parameter to true or provide the 'smaller_than' parameter"
    if two_tailed:
        hypothesis = f"$H_0: \mu = {hypothesized_mean}, H_1: \mu \\neq {hypothesized_mean}, \\alpha = {alpha}$"
        result = stats.ttest_1samp(x, popmean=hypothesized_mean, alternative='two-sided')
        critical_values = stats.distributions.t.interval(1-alpha, result.df)
    else:
        if smaller_than:
            hypothesis = f"$H_0: \mu \leq {hypothesized_mean}, H_1: \mu \\gt {hypothesized_mean}, \\alpha = {alpha}$"
            result = stats.ttest_1samp(x, popmean=hypothesized_mean, alternative='less')
            c, _ = stats.distributions.t.interval(1-(alpha*2), result.df)
            critical_values = (c,)
        else:
            hypothesis = f"$H_0: \mu \geq {hypothesized_mean}, H_1: \mu \\lt {hypothesized_mean}, \\alpha = {alpha}$"
            result = stats.ttest_1samp(x, popmean=hypothesized_mean, alternative='greater')
            _, c = stats.distributions.t.interval(1-(alpha*2), result.df)
            critical_values = (c,)
    display_results(hypothesis, "One sample t-test", critical_values_string(critical_values), f"$t={result.statistic}$", conclude(critical_values, result.statistic, two_tailed=two_tailed, smaller_than=smaller_than))
    display(Markdown(f"*Additional results: $p = {result.pvalue}$*"))
    return result

### Examples

In [145]:
#Example 1
values = np.array([5,4,2,4,5,2,3,3,2,1,4,5,5,4,5])
one_sample_t_test(values, 4, two_tailed=True, alpha=0.05)

**Hypothesis:** $H_0: \mu = 4, H_1: \mu \neq 4, \alpha = 0.05$

**Suitable Test:** One sample t-test

**Critical value(-s):** $C_1 = -2.1447866879169273, C_2 = 2.1447866879169273$

**Test statistic:** $t=-1.1456439237389595$

**Conclusion:** Since $C_1 = -2.1447866879169273 \leq -1.1456439237389595 \leq 2.1447866879169273 = C_2$ we cannot reject $H_0$

*Additional results: $p = 0.27114900383167007$*

TtestResult(statistic=-1.1456439237389595, pvalue=0.27114900383167007, df=14)

In [146]:
#Example 2
values = np.array([5,4,2,4,5,2,3,3,2,1,4,5,5,4,5])
one_sample_t_test(values, 4, two_tailed=False, smaller_than=True, alpha=0.05)

**Hypothesis:** $H_0: \mu \leq 4, H_1: \mu \gt 4, \alpha = 0.05$

**Suitable Test:** One sample t-test

**Critical value(-s):** $C = -1.7613101357748566$

**Test statistic:** $t=-1.1456439237389595$

**Conclusion:** Since $-1.1456439237389595 \nless -1.7613101357748566 = C$ we cannot reject $H_0$

*Additional results: $p = 0.13557450191583503$*

TtestResult(statistic=-1.1456439237389595, pvalue=0.13557450191583503, df=14)

### Calculate

In [ ]:
values = np.array([1,2,3])
hypothesized_mean = 1
one_sample_t_test(values, hypothesized_mean, two_tailed=, smaller_than=, alpha=)

## Two sample t-test

Objective: Test hypothesis that compare population means between two independent samples from different populations

Key questions:
    
- Two-tailed: Do the population means $\mu_1$ and $\mu_2$ differ?
    
- One-tailed: Is the population mean $\mu_1$ smaller or larger than population mean $\mu_2$?
    
Hypotheses (two-tailed):

- $H_0: \mu_1 = \mu_2$

- $H_1: \mu_1 \neq \mu_2$

Test statistic: $t=\frac{\bar{x_1}-\bar{x_2}}{\sqrt{  \frac{(n_1+n_2)[(n_1-1)s_1^2+(n_2-1)s_2^2]}{n_1 * n_2 (n_1+n_2-2)}  }}$ , with sample means $\bar{x_1}$ and $\bar{x_2}$, numbers of samples $n_1$ and $n_2$

If $H_0$ is true, the test statistic follows a t distribution with $df = n_1 + n_2 - 2$

In [152]:
def two_sample_t_test(x_1: np.ndarray, x_2: np.ndarray, two_tailed: bool = True, smaller_than: bool = None, alpha: float = 0.05):
    assert 0 < alpha < 1, "Aplha has to be between 0 and 1"
    assert not two_tailed or (smaller_than == None), "You cannot do a two tailed t-test and provide a value for the 'smaller_than' parameter"
    assert two_tailed or (smaller_than != None), "You have to either set the 'two_tailed' parameter to true or provide the 'smaller_than' parameter"
    df = len(x_1) + len(x_2) - 2
    if two_tailed:
        hypothesis = f"$H_0: \mu_1 = \mu_2, H_1: \mu_1 \\neq \mu_2, \\alpha = {alpha}$"
        result = stats.ttest_ind(x_1, x_2, alternative='two-sided')
        critical_values = stats.distributions.t.interval(1-alpha, df)
    else:
        if smaller_than:
            hypothesis = f"$H_0: \mu_1 \leq \mu_2, H_1: \mu_1 \\gt \mu_2, \\alpha = {alpha}$"
            result = stats.ttest_ind(x_1, x_2, alternative='less')
            c, _ = stats.distributions.t.interval(1-(alpha*2), df)
            critical_values = (c,)
        else:
            hypothesis = f"$H_0: \mu_1 \geq \mu_2, H_1: \mu_1 \\lt \mu_2, \\alpha = {alpha}$"
            result = stats.ttest_ind(x_1, x_2, alternative='greater')
            _, c = stats.distributions.t.interval(1-(alpha*2), df)
            critical_values = (c,)
    display_results(hypothesis, "Two sample t-test", critical_values_string(critical_values), f"$t={result.statistic}$", conclude(critical_values, result.statistic, two_tailed=two_tailed, smaller_than=smaller_than))
    display(Markdown(f"*Additional results:    $p = {result.pvalue}$    $df={df}$*"))
    return result

### Examples

In [153]:
x_1 = np.array([13.3, 6.0, 20, 8,14, 19,18, 25, 16, 24, 15, 1, 15])
x_2 = np.array([22, 16, 21.7, 21, 30, 26, 12, 23.2, 28, 23])
two_sample_t_test(x_1, x_2, two_tailed=True)

**Hypothesis:** $H_0: \mu_1 = \mu_2, H_1: \mu_1 \neq \mu_2, \alpha = 0.05$

**Suitable Test:** Two sample t-test

**Critical value(-s):** $C_1 = -2.079613844727662, C_2 = 2.079613844727662$

**Test statistic:** $t=-2.799960442832919$

**Conclusion:** Reject $H_0$ because $-2.799960442832919 (\lt|\gt) (-2.079613844727662|2.079613844727662) = (C_1|C_2)$

*Additional results: 	$p = 0.010730607904197957$
	$df=21$*

Ttest_indResult(statistic=-2.799960442832919, pvalue=0.010730607904197957)

### Calculations

In [ ]:
x_1 = np.array([1, 2, 3])
x_2 = np.array([1, 2, 3])
two_sample_t_test(x_1, x_2, two_tailed=, smaller_than=, alpha=)

## Test for proportions
Objective: Test hypothesis about population proportion $\theta$ based on one sample

Key questions:
    
- Two-tailed: Does population proportion $\theta$ deviate from a hypothesized proportion $\theta_0$?
    
- One-tailed: Is population proportion $\theta$ larger or smaller than a hypothesized proportion $\theta_0$?
    
Hypotheses (two-tailed):

- $H_0: \theta = \theta_0$

- $H_1: \theta \neq \theta_0$

Test statistic: $z = \frac{P - \theta_0}{\sqrt{  \frac{\theta_0 * (1 - \theta_0)}{n}  }}$ , with observed proportion $P$

If $H_0$ is true and it holds that $n*\theta_0(1-\theta_0) \geq 9$, the test statistic is approximately normally distributed. Otherwise, an exact approach using the binomial distribution is needed.

In [170]:
def one_sample_test_of_proportions(P: float, n: int, population_proportion: float, two_tailed: bool = True, smaller_than: bool = None, alpha: float = 0.05):
    assert 0 < alpha < 1, "Aplha has to be between 0 and 1"
    assert not two_tailed or (smaller_than == None), "You cannot do a two tailed t-test and provide a value for the 'smaller_than' parameter"
    assert two_tailed or (smaller_than != None), "You have to either set the 'two_tailed' parameter to true or provide the 'smaller_than' parameter"
    assert 0 < P < 1, "'P' has to be between 0 and 1"
    assert 0 < population_proportion < 1, "'population_proportion' has to be between 0 and 1"
    
    z_val = (P-population_proportion)/(np.sqrt((population_proportion * (1-population_proportion))/n))
    z_prob = stats.distributions.norm.cdf(-np.abs(z_val))
    
    
    if two_tailed:
        hypothesis = f"$H_0: \\theta = \\theta_0, H_1: \\theta \\neq \\theta_0, \\alpha = {alpha}$"
        critical_values = stats.distributions.norm.interval(1-alpha)
        z_prob *= 2
    else:
        if smaller_than:
            hypothesis = f"$H_0: \\theta \leq \\theta_0, H_1: \\theta \\gt \\theta_0, \\alpha = {alpha}$"
            c, _ = stats.distributions.norm.interval(1-(alpha*2))
        else:
            hypothesis = f"$H_0: \\theta \geq \\theta_0, H_1: \\theta \\lt \\theta_0, \\alpha = {alpha}$"
            _, c = stats.distributions.norm.interval(1-(alpha*2))
        critical_values = (c,)
    display_results(hypothesis, "Test of proportions", critical_values_string(critical_values), f"$z={z_val}$", conclude(critical_values, z_val, two_tailed=two_tailed, smaller_than=smaller_than))
    display(Markdown(f"*Additional results:    $p = {z_prob}$*"))
    display(Markdown(f"<span style='color: #ff0000'>Attention: $n * \\theta_0*(\\theta_0-1) \geq 9$ needed for normal distribution! **Here: {n*population_proportion*(1-population_proportion)}**</span>"))
    return z_val, z_prob

### Examples

In [175]:
one_sample_test_of_proportions(.26, 340, 1/3, two_tailed=False, smaller_than=True)

**Hypothesis:** $H_0: \theta \leq \theta_0, H_1: \theta \gt \theta_0, \alpha = 0.05$

**Suitable Test:** Test of proportions

**Critical value(-s):** $C = -1.6448536269514729$

**Test statistic:** $z=-2.868449058289164$

**Conclusion:** Reject $H_0$ because $t = -2.868449058289164 \lt -1.6448536269514729 = C$

*Additional results:    $p = 0.0020624480892897866$*

<span style='color: #ff0000'>Attention: $n * \theta_0*(\theta_0-1) \geq 9$ needed for normal distribution! **Here: 75.55555555555556**</span>

(-2.868449058289164, 0.0020624480892897866)

### Calculations

In [ ]:
P = 0.1
n = 100
population_proportion = 1/6
one_sample_test_of_proportions(P, n, population_proportion, two_tailed=, smaller_than=, alpha=)

## Two sample test for proportions
Objective: Test hypotheses that compare population proportions between two independent samples from different populations

Key questions:
    
- Two-tailed: Do population proportions $\theta_1$ and $\theta_2$ differ?
    
- One-tailed: Is population proportion $\theta_1$ larger or smaller than population proportion $\theta_2$?
    
Hypotheses (two-tailed):

- $H_0: \theta_1 = \theta_2$

- $H_1: \theta_1 \neq \theta_2$

Test statistic: $z = \frac{P_1 - P_2}{\sqrt{P(1-P)*( \frac{1}{n_1} + \frac{1}{n_2} )}}$ , where $P = \frac{P_1n_1+P_2n_2}{n_1+n_2}$

If $H_0$ is true and $n_1,n_2 \geq 30$, the test statistic is approximately normally distributed.

In [171]:
def two_sample_test_of_proportions(x_1: int, x_2: int, n_1: int, n_2, two_tailed: bool = True, smaller_than: bool = None, alpha: float = 0.05):
    assert 0 < alpha < 1, "Aplha has to be between 0 and 1"
    assert not two_tailed or (smaller_than == None), "You cannot do a two tailed t-test and provide a value for the 'smaller_than' parameter"
    assert two_tailed or (smaller_than != None), "You have to either set the 'two_tailed' parameter to true or provide the 'smaller_than' parameter"
    assert type(x_1) == type(x_2), "Both parameters 'x_1' and 'x_2' have to be of the same type integer or float"
    assert type(x_1) == int or type(x_1) == float, "Parameters 'x_1' and 'x_2' have to be of int or float type"
    
    if type(x_1) == float:
        assert 0 <= x_1 <= 1 and 0 <= x_2 <= 1, "Float parameters 'x_1' and 'x_2' have to be between zero and one"
        x_1 *= n_1
        x_2 *= n_2
    
    avg_p = (x_1 + x_2) / (n_1 + n_2)
    z_val = (x_1/n_1 - x_2/n_2) / np.sqrt(avg_p * (1-avg_p) * (1/n_1 + 1/n_2))
    z_prob = stats.distributions.norm.cdf(-np.abs(z_val))
    
    
    if two_tailed:
        hypothesis = f"$H_0: \\theta_1 = \\theta_2, H_1: \\theta_1 \\neq \\theta_2, \\alpha = {alpha}$"
        critical_values = stats.distributions.norm.interval(1-alpha)
        z_prob *= 2
    else:
        if smaller_than:
            hypothesis = f"$H_0: \\theta_1 \leq \\theta_2, H_1: \\theta_1 \\gt \\theta_2, \\alpha = {alpha}$"
            c, _ = stats.distributions.norm.interval(1-(alpha*2))
        else:
            hypothesis = f"$H_0: \\theta_1 \geq \\theta_2, H_1: \\theta_1 \\lt \\theta_2, \\alpha = {alpha}$"
            _, c = stats.distributions.norm.interval(1-(alpha*2))
        critical_values = (c,)
    display_results(hypothesis, "Two sample test of proportions", critical_values_string(critical_values), f"$z={z_val}$", conclude(critical_values, z_val, two_tailed=two_tailed, smaller_than=smaller_than))
    display(Markdown(f"*Additional results:    $p = {z_prob}$*"))
    display(Markdown("<span style='color: #ff0000'>Attention: $n_1,n_2 \geq 30$ needed for normal distribution!</span>"))
    return z_val, z_prob

### Examples

In [166]:
two_sample_test_of_proportions(.26, .19, 252, 214, two_tailed=False, smaller_than=False)

**Hypothesis:** $H_0: \theta_1 \geq \theta_2, H_1: \theta_1 \lt \theta_2, \alpha = 0.05$

**Suitable Test:** Two sample test of proportions

**Critical value(-s):** $C = 1.6448536269514722$

**Test statistic:** $z=1.7952880715638093$

**Conclusion:** Reject $H_0$ because $t = 1.7952880715638093 \gt 1.6448536269514722 = C$

*Additional results:    $p = 0.03630390727849902$*

<span style='color: #ff0000'>Attention: $n_1,n_2 \geq 30$ needed for normal distribution!</span>

(1.7952880715638093, 0.03630390727849902)

In [167]:
two_sample_test_of_proportions(66, 41, 252, 214, two_tailed=False, smaller_than=False)

**Hypothesis:** $H_0: \theta_1 \geq \theta_2, H_1: \theta_1 \lt \theta_2, \alpha = 0.05$

**Suitable Test:** Two sample test of proportions

**Critical value(-s):** $C = 1.6448536269514722$

**Test statistic:** $z=1.798518944675899$

**Conclusion:** Reject $H_0$ because $t = 1.798518944675899 \gt 1.6448536269514722 = C$

*Additional results:    $p = 0.03604740462220061$*

<span style='color: #ff0000'>Attention: $n_1,n_2 \geq 30$ needed for normal distribution!</span>

(1.798518944675899, 0.03604740462220061)

### Calculations

In [ ]:
x_1 = 20   # Either number of positives as integer
x_2 = 0.25 # Or proportian as float between 0 and 1
n_1 = 80
n_2 = 100
two_sample_test_of_proportions(x_1, x_2, n_1, n_2, two_tailed=, smaller_than=, alpha=)

## t-test for independence
Objective: Test hypotheses about population correlation $\rho$

Key questions:
    
- Two-tailed: Is the population correlation $\rho$ differnt from $0$?
    
- One-tailed: Is the population correlation $\rho$ smaller or larger than $0$?
    
Hypotheses (two-tailed):

- $H_0: \rho = 0$

- $H_1: \rho \neq 0$

Test statistic: $t = \frac{r}{\sqrt{1-r^2}}\sqrt{n-2}$ , with pearsons $r$

If $H_0$ is true, the test statistic follows a t-distribution with $df = n - 2$

In [182]:
def t_test_for_independence(x_1: np.ndarray, x_2: np.ndarray, two_tailed: bool = True, smaller_than: bool = None, alpha: float = 0.05):
    assert 0 < alpha < 1, "Aplha has to be between 0 and 1"
    assert not two_tailed or (smaller_than == None), "You cannot do a two tailed t-test and provide a value for the 'smaller_than' parameter"
    assert two_tailed or (smaller_than != None), "You have to either set the 'two_tailed' parameter to true or provide the 'smaller_than' parameter"
    
    df = len(x_1) - 2
    
    if two_tailed:
        r, p = stats.pearsonr(x_1, x_2, alternative='two-sided')
        hypothesis = f"$H_0: \\rho = 0, H_1: \\rho \\neq 0, \\alpha = {alpha}$"
        critical_values = stats.distributions.t.interval(1-alpha, df)
    else:
        if smaller_than:
            r, p = stats.pearsonr(x_1, x_2, alternative='less')
            hypothesis = f"$H_0: \\rho \leq 0, H_1: \\rho \\gt 0, \\alpha = {alpha}$"
            c, _ = stats.distributions.t.interval(1-(alpha*2), df)
        else:
            r, p = stats.pearsonr(x_1, x_2, alternative='greater')
            hypothesis = f"$H_0: \\rho \geq 0, H_1: \\rho \\lt 0, \\alpha = {alpha}$"
            _, c = stats.distributions.t.interval(1-(alpha*2), df)
        critical_values = (c,)
    
    t_val = r/(np.sqrt(1-r**2))*np.sqrt(len(x_1)-2)
    
    display_results(hypothesis, "t-test for independence", critical_values_string(critical_values), f"$t={t_val}$", conclude(critical_values, t_val, two_tailed=two_tailed, smaller_than=smaller_than))
    display(Markdown(f"*Additional results:    $p = {p}$    $df= {df}$*"))
    return t_val, p

### Examples

In [183]:
x_1 = np.array([312, 462, 419, 433, 442, 305, 509, 287, 245, 457])
x_2 = np.array([402, 456, 432, 476, 419, 340, 463, 360, 280, 500])
t_test_for_independence(x_1, x_2, two_tailed=True)

**Hypothesis:** $H_0: \rho = 0, H_1: \rho \neq 0, \alpha = 0.05$

**Suitable Test:** t-test for independence

**Critical value(-s):** $C_1 = -2.3060041350333704, C_2 = 2.3060041350333704$

**Test statistic:** $t=5.783268809989026$

**Conclusion:** Reject $H_0$ because $5.783268809989026 (\lt|\gt) (-2.3060041350333704|2.3060041350333704) = (C_1|C_2)$

*Additional results:    $p = 0.00041297215588927003$    $df= 8$*

(5.783268809989026, 0.00041297215588927003)

### Calculations

In [ ]:
x_1 = np.array([1,2,3])
x_2 = np.array([1,2,3])
t_test_for_independence(x_1, x_2, two_tailed=, small_than=,alpha=)

## $X^2$ test for goodness of fit
Objective: Test hypotheses about whether a sample distribution of a categorical variable follows a hypothesized population distribution

Key question:
    
- Is the distribution observed in the sample consistent with the distribution that is hypothesized to exist in the population?
    
Hypotheses (two-tailed logic only):

- $H_0:$ Sample distribution and population distribution are consistent

- $H_1:$ Sample distribution and population distribution are inconsistent

Test statistic: $\chi = \sum_{i=1}^k \frac{(h_i-n * p_i)^2}{n*p_i}$ , with 

- $h_i:$ empirically observed frequency of category $i$
- $p_i:$ probability from hypothesized distribution to observe category $i$
- $n:$ sample size


If $H_0$ is true, the test statistic is $X^2$-distributed with $df = k - 1$

**Use Only of for all categories $n*p_i \geq 5$, otherwise merge categories**

In [186]:
def chi_square_test_goodness_of_fit(observed_frequencies: np.ndarray, expected_frequencies: np.ndarray = None, alpha: float = 0.05):
    assert 0 < alpha < 1, "Aplha has to be between 0 and 1"
    assert (expected_frequencies == None) or len(observed_frequencies) == len(expected_frequencies), "observed and expected frequencies shoudl have the same dimensions"
    
    # TODO: Check if categories have to be merged
    
    hypothesis = f"$H_0:$ Sample distribution and population distribution are consistent, $H_1:$ Sample distribution and population distribution are inconsistent, $\\alpha = {alpha}$"
    df = len(observed_frequencies) - 1
    X2, p = stats.chisquare(observed_frequencies, expected_frequencies)
    critical_values = stats.distributions.chisquare.interval(1-alpha, df)
    
    display_results(hypothesis, "Chi-square test for goodness of fit", critical_values_string(critical_values), f"$\chi^2={X2}$", conclude(critical_values, X2, two_tailed=True))
    display(Markdown(f"*Additional results: $p = {p}$*"))
    return X2, p

### Examples

### Calculations

## $X^2$ test for independence
Objective: Test hypotheses whether two categorical (nominally scaled) variables are independent in the population

Key question:
    
- Are two nominally scaled variables independent of each other?

Input:

- Contingency table showing observed frequencies of both variables A and B with their categories $A_i (i=1,...,r)$ and $B_j (j = 1,...,s)$
    
Hypotheses (two-tailed logic only):

- $H_0:$ Both variables are independent from each other

- $H_1:$ Both variables are not independent from each other

Test statistic: $\chi = \sum_{i=1}^r\sum_{j=1}^s \frac{(h_{ij}-n*p_i*p_j)^2}{n*p_i*p_j}$ , with 

- $h_ij:$ empirically observed frequency of cell $ij$
- $p_i:$ overall empirical relative frequency of observing row $i$
- $p_j:$ overall empirical relative frequency of observing column $j$
- $n:$ sample size (overall)


If $H_0$ is true, the test statistic is $X^2$-distributed with $df = (r - 1)*(s-1)$

**Use Only of for all categories $n*p_i*p_j \geq 5$, otherwise merge categories**

In [ ]:
def z_test_2_sample_proportions(x1, x2, n1, n2, two_tailed=True):
    '''
    Calculate the test statistic for a z-test on 2 proportions from independent samples
    x1, x2: number of successes in group 1 and 2
    n1, n2: total number of observations in group 1 and 2
    Returns: test statistic (z), and p-value 
    '''
    avg_p = (x1 + x2) / (n1 + n2)
    z_val = (x1/n1 - x2/n2) / np.sqrt(avg_p * (1-avg_p) * (1/n1 + 1/n2))
    z_prob = stats.distributions.norm.cdf(-np.abs(z_val))

    if two_tailed:
        return z_val, 2*z_prob

    else:
        return z_val, z_prob

In [6]:
z_test_2_sample_proportions(.26*252, .19*214, 252, 214, two_tailed=False)

(1.7952880715638093, 0.03630390727849902)

In [8]:
z_test_2_sample_proportions(.26*50, .19*50, 50, 50, two_tailed=False)

(0.8381580761249496, 0.2009709646021316)

In [172]:
def z_test_1_sample_proportions(P, n, sigma_0, two_tailed=True):
    '''
    Returns: test statistic (z), and p-value 
    '''
    z_val = (P-sigma_0)/(np.sqrt((sigma_0 * (1-sigma_0))/n))
    z_prob = stats.distributions.norm.cdf(-np.abs(z_val))

    if two_tailed:
        return z_val, 2*z_prob

    else:
        return z_val, z_prob

In [10]:
z_test_1_sample_proportions(.26, 340, 1/3, two_tailed=False)

(-2.868449058289164, 0.0020624480892897866)

In [13]:
(1/3)+(np.sqrt((1/3)*(2/3)/340)*-1.6449)

0.291280642994832